In [2]:
!git clone https://github.com/MinkaiXu/CGCF-ConfGen

Cloning into 'CGCF-ConfGen'...
remote: Enumerating objects: 74, done.
remote: Counting objects: 100% (74/74), done.
remote: Compressing objects: 100% (71/71), done.
remote: Total 74 (delta 25), reused 34 (delta 1), pack-reused 0
Unpacking objects: 100% (74/74), done.


In [3]:
!pip install kora -q
import kora.install.rdkit

     |████████████████████████████████| 57 kB 5.4 MB/s 
     |████████████████████████████████| 56 kB 5.8 MB/s 


In [4]:
pip install torch==1.6.0

     |████████████████████████████████| 748.8 MB 18 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
      Successfully uninstalled torch-1.10.0+cu111
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.11.1+cu111 requires torch==1.10.0, but you have torch 1.6.0 which is incompatible.
torchtext 0.11.0 requires torch==1.10.0, but you have torch 1.6.0 which is incompatible.
torchaudio 0.10.0+cu111 requires torch==1.10.0, but you have torch 1.6.0 which is incompatible.


In [5]:
!pip install torchvision==0.7.0

     |████████████████████████████████| 5.9 MB 29.5 MB/s 
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.11.1+cu111
    Uninstalling torchvision-0.11.1+cu111:
      Successfully uninstalled torchvision-0.11.1+cu111


In [6]:
pip install torchdiffeq==0.0.1

In [7]:
pip install tqdm networkx scipy scikit-learn h5py tensorboard 

In [8]:
pip install --no-index torch-scatter -f https://pytorch-geometric.com/whl/torch-1.6.0+cu101.html

Looking in links: https://pytorch-geometric.com/whl/torch-1.6.0+cu101.html
     |████████████████████████████████| 2.8 MB 701 kB/s 


In [9]:
pip install --no-index torch-sparse -f https://pytorch-geometric.com/whl/torch-1.6.0+cu101.html


Looking in links: https://pytorch-geometric.com/whl/torch-1.6.0+cu101.html
     |████████████████████████████████| 1.6 MB 547 kB/s 


In [10]:
pip install --no-index torch-cluster -f https://pytorch-geometric.com/whl/torch-1.6.0+cu101.html

Looking in links: https://pytorch-geometric.com/whl/torch-1.6.0+cu101.html
     |████████████████████████████████| 1.1 MB 34.6 MB/s 


In [11]:
pip install --no-index torch-spline-conv -f https://pytorch-geometric.com/whl/torch-1.6.0+cu101.html

Looking in links: https://pytorch-geometric.com/whl/torch-1.6.0+cu101.html
     |████████████████████████████████| 367 kB 556 kB/s 


In [12]:
pip install torch-geometric

     |████████████████████████████████| 370 kB 25.6 MB/s 
     |████████████████████████████████| 482 kB 52.3 MB/s 
     |████████████████████████████████| 41 kB 726 kB/s 
  Created wheel for torch-geometric: filename=torch_geometric-2.0.3-py3-none-any.whl size=581968 sha256=2560e1935fde8d1514ce260d048631fde0c093cf15170706a33ecb118ab79a71
  Stored in directory: /root/.cache/pip/wheels/c3/2a/58/87ce0508964d4def1aafb92750c4f3ac77038efd1b9a89dcf5
Successfully built torch-geometric


In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
cd CGCF-ConfGen/

/content/CGCF-ConfGen


In [30]:
#@title import packages
import os
import argparse
import torch
import torch.utils.tensorboard
from torch_geometric.data import DataLoader
from torch_geometric.loader import DataLoader
from tqdm.auto import tqdm

from models.edgecnf import *
from models.cnf_edge import NONLINEARITIES, LAYERS, SOLVERS
from utils.dataset import *
from utils.transforms import *
from utils.misc import *

In [16]:
#@title Arguments
parser = argparse.ArgumentParser()
# BEGIN
# Model arguments
parser.add_argument('--activation', type=str, default='softplus')
parser.add_argument('--hidden_dim', type=int, default=64)
parser.add_argument("--num_blocks", type=int, default=1,
                    help='Number of stacked CNFs.')
parser.add_argument("--layer_type", type=str, default="concatsquash", choices=LAYERS)
parser.add_argument('--time_length', type=float, default=0.5)
parser.add_argument('--train_T', type=eval, default=True, choices=[True, False])
parser.add_argument('--use_adjoint', type=eval, default=True, choices=[True, False])
parser.add_argument('--solver', type=str, default='dopri5', choices=SOLVERS)
parser.add_argument('--atol', type=float, default=1e-5)
parser.add_argument('--rtol', type=float, default=1e-5)
parser.add_argument('--batch_norm', type=eval, default=True, choices=[True, False])
parser.add_argument('--sync_bn', type=eval, default=False, choices=[True, False])
parser.add_argument('--bn_lag', type=float, default=0)
parser.add_argument('--spectral_norm', type=eval, default=True, choices=[True, False])
parser.add_argument('--train_noise_std', type=float, default=0.1)

# Datasets and loaders
parser.add_argument('--aux_edge_order', type=int, default=3)
parser.add_argument('--train_dataset', type=str, default='./data/qm9/train.pkl')
parser.add_argument('--val_dataset', type=str, default='./data/qm9/val.pkl')
parser.add_argument('--train_batch_size', type=int, default=128)
parser.add_argument('--val_batch_size', type=int, default=256)
parser.add_argument('--num_workers', type=int, default=8)
parser.add_argument('--max_val_batch', type=int, default=5)

# Optimizer and scheduler
parser.add_argument('--lr', type=float, default=1e-3)
parser.add_argument('--weight_decay', type=float, default=0)
parser.add_argument('--sched_factor', type=float, default=0.5)
parser.add_argument('--sched_patience', type=int, default=3,
                    help='Patience steps = sched_patience * val_freq')
parser.add_argument('--sched_min_lr', type=int, default=1e-5)
parser.add_argument('--beta1', type=float, default=0.95)
parser.add_argument('--beta2', type=float, default=0.999)

# Training
parser.add_argument('--seed', type=int, default=2020)
parser.add_argument('--logging', type=eval, default=True, choices=[True, False])
parser.add_argument('--device', type=str, default='cuda')
parser.add_argument('--max_iters', type=int, default=50*1000, 
                    help='Max iterations for MLE pre-training of CNF')
parser.add_argument('--val_freq', type=int, default=300)
parser.add_argument('--inspect_freq', type=int, default=50)
parser.add_argument('--tag', type=str, default='')
parser.add_argument('--resume', type=str, default=None)
parser.add_argument('--log_root', type=str, default='./logs')
# END
args = parser.parse_args(['--train_dataset','/content/drive/MyDrive/test_QM9.pkl','--val_dataset','/content/drive/MyDrive/val_QM9.pkl'])
seed_all(args.seed)

In [17]:
tf = get_standard_transforms(order=args.aux_edge_order)

In [18]:
train_dset = MoleculeDataset(args.train_dataset, transform=tf)

In [31]:
train_iterator = get_data_iterator(DataLoader(train_dset, batch_size=args.train_batch_size, shuffle=True, drop_last=True))

In [20]:
args.device

'cuda'

In [21]:
model = EdgeCNF(args).to(args.device)

In [32]:
batch = next(train_iterator).to(args.device)

RuntimeError: ignored

In [22]:
import torch

from .common import *
from .cnf_edge import CNF, ODEfunc, ODEgnn, MovingBatchNorm1d, SequentialFlow, count_nfe, add_spectral_norm, spectral_norm_power_iteration
from .distgeom import *

ImportError: ignored

In [28]:
!python train.py --train_dataset /content/drive/MyDrive/train_QM9.pkl --val_dataset /content/drive/MyDrive/val_QM9.pkl

[2022-01-16 03:31:45,040::train::INFO] Namespace(activation='softplus', atol=1e-05, aux_edge_order=3, batch_norm=True, beta1=0.95, beta2=0.999, bn_lag=0, device='cuda', hidden_dim=64, inspect_freq=50, layer_type='concatsquash', log_root='./logs', logging=True, lr=0.001, max_iters=50000, max_val_batch=5, num_blocks=1, num_workers=8, resume=None, rtol=1e-05, sched_factor=0.5, sched_min_lr=1e-05, sched_patience=3, seed=2020, solver='dopri5', spectral_norm=True, sync_bn=False, tag='', time_length=0.5, train_T=True, train_batch_size=128, train_dataset='/content/drive/MyDrive/train_QM9.pkl', train_noise_std=0.1, use_adjoint=True, val_batch_size=256, val_dataset='/content/drive/MyDrive/val_QM9.pkl', val_freq=300, weight_decay=0)
[2022-01-16 03:31:45,040::train::INFO] Loading dataset...
100% 50000/50000 [01:06<00:00, 750.64it/s]
100% 2000/2000 [00:02<00:00, 783.22it/s]
/usr/local/lib/python3.7/dist-packages/torch_geometric/deprecation.py:13: UserWarning: 'data.DataLoader' is deprecated, use 'l

In [25]:
args.train_dataset

'/content/drive/MyDrive/test_QM9.pkl'

In [29]:
torch.__version__

'1.6.0'